<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Оценка периода полураспада Ba-137m

### Examples – Particle and Nuclear Physics
<section class="post-meta">
By Jonas Tjemsland, Andreas S. Krogen and Jon Andreas Støvneng.
</section>
Last edited: January 21st 2019
___

## Вступление
Cs-137, как и ряд других радиоактивных изотопов, был выброшен в окружающую среду почти во время всех испытаний ядерного оружия и некоторых ядерных аварий, наиболее заметных из которых была Чернобыльская катастрофа (1986) и катастрофа на АЭС "Фукусима-Дайити".(2011) [[1]](#rsc). Исходный изотоп Cs-137 с периодом полураспада 30.1 года бета распадается на Ba-137m. Это метастабильное состояние, которое далее распадается при гамма-излучении 662 кэВ с периодом полураспада приблизительно 2.55 минуты до стабильного изотопа Ba-137 [[2]](#rsc).

В этом блокноте мы будем использовать подгонку кривой для оценки периода полураспада изотопа Ba-137m. Основная цель этого блокнота - представить несколько функций подгонки кривых от SciPy и NumPy. Данные основаны на наборе измерений, проведенных в лабораторных условиях в NTNU в 2016 году.

## Экспоненциальный закон радиоактивного распада

Радиоактивность - это спонтанный процесс, при котором ядро нестабильного изотопа теряет энергию, испуская излучение. Это излучение включает альфа-частицы, бета-частицы (электрон или позитрон), захват электронов и гамма-излучение.

Основное предположение статистической теории радиоактивного распада состоит в том, что вероятность в единицу времени распада атома постоянна, независимо от возраста атома [[3]](#rsc). То есть,

\begin{equation}
    \lambda=-\frac{\text{d}N/\text{d}t}{N},
\end{equation}

где $N$ - количество ядер, присутствующих в момент времени $t$. $\lambda$ часто называют константой распада. Решение этого дифференциального уравнения дает экспоненциальный закон радиоактивного распада,

\begin{equation}
    N(t)=N_0e^{-\lambda t},
\end{equation}

где $N_0=N(t=0)$. Период полураспада $t_{1/2}$ - это время, необходимое для того, чтобы активность данного количества радиоактивного вещества распалась до половины его первоначального значения. Поскольку активность пропорциональна количеству радиоактивных ядер, период полураспада становится

\begin{equation}
   t_{1/2}=\frac{\ln(2)}{\lambda}.
\end{equation}

Экспоненциальный закон радиоактивного распада не учитывает систематические ошибки и фоновое излучение. Простой способ ввести это в нашу модель - наложить наше уравнение на постоянный фон. То есть,

\begin{equation}
\tilde N(t)=N_0e^{-\lambda t} + c,
\end{equation}

для некоторой постоянной $c$.

## Несколько замечаний по измерениям

При измерениях использовался NaI-сцинтиллятор. Кристалл NaI-сцинтиллятора поглощает излучение гамма-фотонов, образующееся при распаде Ba-137m, и излучает видимый свет, пропорциональный энергии гамма-фотона. Трубка для умножения фотонов поглощает световые фотоны и испускает импульс напряжения, пропорциональный количеству видимого света. Измерения состояли в определении количества отсчетов (каждого события гамма-поглощения) за заданный промежуток времени. Каждое измерение длилось 10 секунд. Обратите внимание, что на самом деле мы измеряем активность $A(t)$ (количество отсчетов в секунду), но общая формула, приведенная выше, не меняется.

Измеренные значения приведены в следующем фрагменте кода.

In [ ]:
# Number of counts in each measurement (10 seconds)
counts_Ba = [ 
 80412., 76400., 73071., 69203., 65735., 62529., 59948., 56926.,
 54243., 51490., 49255., 46256., 44152., 42339., 40003., 38525.,
 36419., 35059., 33011., 31439., 29971., 28300., 27179., 25554.,
 24475., 23143., 22187., 21279., 19845.
            ]
# Start time of measurement
times_Ba  = [
     0.,    12.,    23.,    34.,    45.,    57.,    68.,    79.,
     91.,  102.,   113.,   124.,   136.,   147.,   158.,   169.,
    181.,   192.,  203.,   214.,   226.,   237.,   248.,   259.,
    271.,   282.,  293.,   305.,   316.
            ]

## Оценка периода полураспада

Чтобы оценить период полураспада $\lambda_\text{Ba}$ Ba-137m, нам нужно сопоставить экспоненциальный закон радиоактивности с нашими данными. Мы начинаем с импорта некоторых необходимых пакетов, установки общих параметров фигуры и определения функции экспоненциального закона распада.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set some figure parameters
newparams = {'figure.figsize': (12, 6), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 3,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

def func(t, N0, lambd, c):
    """ Callable model function to be used by the curve fitting
    functions. It must take the independent variable as the
    first argument and the parameters to fit as separate remaining
    arguments.
    """
    return N0 * np.exp(-lambd * np.array(t)) + c

### curve_fit()
Мы начнем с использования [scipy.optimize.curve_fit()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), который использует нелинейные наименьшие квадраты для подгонки произвольной функции к данным. Функция имеет три входных параметра: функция модели, измеренные $x$-данные (*times_Ba*) и измеренные $y$-данные (*counts_Ba*). Есть также несколько необязательных аргументов, но это не относится к данному уроку. Функция возвращает массив оптимальных значений параметров (*popt*) и соответствующую ковариационную матрицу (*pcov*). Диагонали матрицы ковариации обеспечивают дисперсию оценок параметров. Стандартное отклонение - это квадратный корень из дисперсии.

In [ ]:
from scipy.optimize import curve_fit

Обратите внимание, что нам нужно взять экспоненту от $\lambda\cdot t$, где $\lambda\approx 150\;\text{s}^{-1}$ и $t\leq 316\;\text{s}$. Другими словами, нам нужно вычислить $\exp(150\cdot 316)=\exp(47400)\sim 10^{20585}$ численно. Это большое число, и мы, скорее всего, получим ошибку переполнения. Есть несколько способов обойти это, например, взяв логарифм и выполнив линейную подгонку, или нормализовав данные. Мы нормализуем так, чтобы максимальное время и максимальное количество отсчетов стали единицей.

In [ ]:
max_time = np.max(times_Ba)
max_counts = np.max(counts_Ba)

И вызовем *curve_fit()*!

In [ ]:
# Perform the curve fit on normalized data
popt, pcov = curve_fit(func, times_Ba/max_time, counts_Ba/max_counts)

# Standard deviation
perr = np.sqrt(np.diag(pcov))

# Parameter estimates reexpressed in ordinary units
lambd = popt[1]/max_time
lambd_err = perr[1]/max_time # Standard deviation in lambd
c = popt[2]*max_counts
N0 = popt[0]*max_counts

# Calculate half life with error
half_life = np.log(2)/lambd
half_life_err = half_life*lambd_err/lambd

print("Half life: %.1fs +- %.1fs"%(half_life, half_life_err))

Предполагаемый период полураспада Ba-137m составляет $t_{1/2}=(159.5\pm 2.0)\;\text{s}$. Реальный период распада $153.12\;\text{s}$  не находится в границах ошибок. Существует множество ошибок, которые могли бы объяснить это несоответствие, например, чистота источника и ошибка в обнаружении. Более того, данные были измерены во время короткого лабораторного занятия.

Давайте визуализируем подгонку!

In [ ]:
plt.plot(times_Ba, func(times_Ba, N0, lambd, c)/10,
    label=r'Best fit on the form $N(t)=a\cdot \exp(-\lambda\cdot t) + c$')
plt.plot(times_Ba, np.array(counts_Ba)/10, 'o', label='Measured data')
plt.xlabel(r'$t$, [s]')
plt.ylabel(r'Activity, $A(t)$, [1/s]')
plt.legend()
plt.show()

### polyfit()
Давайте выполним те же вычисления, что и выше, используя линейную регрессию и [numpy.polyfit()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html), который выполняет полиномиальную подгонку по методу наименьших квадратов. Функция имеет три входных параметра: измеренные $x$-данные (*times_Ba*), измеренные $y$-данные (*counts_Ba*) и степень полинома (*1*). Функция возвращает полиномиальные коэффициенты, сначала наивысшая степень. Кроме того, мы будем использовать необязательный аргумент *cov=True*, который указывает функции возвращать матрицу ковариации.

In [ ]:
from numpy import polyfit

Взяв логарифм по экспоненциальному закону радиоактивного распада и допустив $c=0$, мы получим

\begin{equation}
\log(\tilde N(t))=\log(N_0) -\lambda t.
\end{equation}

Таким образом, мы можем выполнить линейную подгонку к логарифму наших данных!

In [ ]:
# Perform linear fit on the logarithm of the measurements
popt, pcov = np.polyfit(times_Ba, np.log(np.array(counts_Ba)/10), 1, cov=True)
perr = np.sqrt(np.diag(pcov))

# Calculate lambda with error
lambd = -popt[0]
lambd_err = perr[0]
# Calculate half life with error
half_life = np.log(2)/lambd
half_life_err = half_life*lambd_err/lambd
# Print results
print("Half life: %.1fs +- %.1fs"%(half_life, half_life_err))

Этот результат примерно такой же, какой мы получили ранее. Однако в данном случае мы не учли систематические ошибки (фон). Давайте визуализируем результаты.

In [ ]:
plt.plot(times_Ba, np.array(times_Ba)*popt[-2] + popt[-1],
        label=r'Best fit')
plt.plot(times_Ba, np.log(np.array(counts_Ba)/10), 'o', label='Measured data')
plt.xlabel(r'$t$, [s]')
plt.ylabel(r'$\log(A(t)/A_0)$')
plt.legend()
plt.show()

___
## Дальнейшая Работа

- Как можно оценить период полураспада изотопа, который распадается на другой нестабильный изотоп? (см., например, ссылку [[4]](#rsc))

<a id="rsc"></a>
## Ресурсы и дальнейшее чтение:

[[1]](): Caesium-137: https://en.wikipedia.org/wiki/Caesium-137, 2016, [Acquired 09.2017]

[[2]](): NATS: Isotope Generator, http://nats-usa.com/wp-content/uploads/2012/11/Isotope-Generator.pdf, [Acquired 09.2017]

[[3]](): Krane, K.: Introductory Nuclear Physics, John Wiley & Sons, 1988

[[4]](): Pommé, S.: The uncertainty of the half-life,  BIPM & IOP Publishing Ltd,  http://iopscience.iop.org/article/10.1088/0026-1394/52/3/S51, 2015, [Acquired 09.2017]